In [1]:
import shap
import pandas as pd
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt
import ast

In [2]:
dataset = pd.read_csv('../dataset/HR_result.csv')

movies = pd.read_csv('../dataset/simpler_movie_dataset.csv')
movies = movies.drop_duplicates(subset='id').set_index('id').to_dict('index')

users = pd.read_csv('../dataset/user_metadata.csv', index_col='userId').to_dict('records')

In [3]:
user_dataset = dataset.copy()
user_dataset['movies'] = user_dataset['movies'].apply(ast.literal_eval)
user_dataset['userId'] = user_dataset['userId'].astype('int')
user_dataset = user_dataset.explode('movies')[['userId', 'movies']]
user_dataset['movies'] = user_dataset['movies'].astype('int')
user_info = user_dataset['userId'].apply(lambda x: users[x]).apply(pd.Series)
mvs_info = user_dataset['movies'].apply(lambda x: movies[x]).apply(pd.Series)
dataset = pd.concat([user_dataset, user_info, mvs_info], axis=1).reset_index()

In [4]:
dataset

,index,userId,movies,total_movies,userAvgRating,userAvgBudget,userTopLanguages,userTopGenres,userTopPH,userAvgPopularity,...,production_countries,revenue,runtime,spoken_languages,video,vote_average,vote_count,movie_age,abstract_col,title
0,0,10143,323695,87,4.0,1500000.0,en,Comedy,Paramount Pictures,8.176863,...,NaN,0,0,NaN,False,4.5,1,2988,music,Devo: Hardcore Live!
1,0,10143,387700,87,4.0,1500000.0,en,Comedy,Paramount Pictures,8.176863,...,NaN,0,0,NaN,False,6.5,2,2598,music,The Passion
2,0,10143,221917,87,4.0,1500000.0,en,Comedy,Paramount Pictures,8.176863,...,Ireland,0,0,English,False,8.0,2,10083,romance castle rock entertainment columbia pic...,The Run of the Country
3,0,10143,37429,87,4.0,1500000.0,en,Comedy,Paramount Pictures,8.176863,...,Turkey,0,0,Türkçe,False,7.5,2,5225,turkey türkçe,Vali
4,0,10143,85304,87,4.0,1500000.0,en,Comedy,Paramount Pictures,8.176863,...,NaN,0,52,English,False,4.3,2,5480,music documentary english,John Lennon - Plastic Ono Band
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6979,387,9989,665,9,3.5,1530000.0,en,Action,Claussen + Wöbke Filmproduktion GmbH,7.284477,...,United States of America,146900000,212,English,False,7.5,660,23137,action adventure history metro-goldwyn-mayer (...,Ben-Hur
6980,387,9989,675,9,3.5,1530000.0,en,Action,Claussen + Wöbke Filmproduktion GmbH,7.284477,...,United Kingdom,938212738,138,English,False,7.4,5633,5786,adventure fantasy family mystery warner bros. ...,Harry Potter and the Order of the Phoenix
6981,387,9989,291805,9,3.5,1530000.0,en,Action,Claussen + Wöbke Filmproduktion GmbH,7.284477,...,Canada,334901337,129,普通话,False,6.7,3362,2524,action adventure comedy crime mystery thriller...,Now You See Me 2
6982,387,9989,767,9,3.5,1530000.0,en,Action,Claussen + Wöbke Filmproduktion GmbH,7.284477,...,United Kingdom,933959197,153,English,False,7.4,5435,5046,adventure fantasy family warner bros. heyday f...,Harry Potter and the Half-Blood Prince


In [5]:
col_list = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'movie_age', 'production_companies', 'production_countries', 'spoken_languages', 'genres', 'total_movies', 'userAvgRating', 'userAvgBudget', 'userAvgPopularity', 'userAvgMovieAge', 'userAvgRuntime', 'userTopPH', 'userTopGenres']
categorical_col = ['production_companies', 'production_countries', 'spoken_languages', 'genres', 'userTopPH', 'userTopGenres']
dataset[categorical_col] = dataset[categorical_col].fillna('other')
dataset[col_list] = dataset[col_list].fillna(-1)

In [6]:
model =CatBoostClassifier()
model.load_model('../dataset/Catboost',           
           format="cbm")

In [16]:
train_dataset = Pool(dataset[col_list], feature_names=col_list, cat_features=categorical_col) 

In [17]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(train_dataset)

In [18]:
shap.plots.bar(shap_values.cohorts(2).abs.mean(0))

TypeError: float() argument must be a string or a real number, not 'Pool'

In [20]:
shap.plots.heatmap(shap_values[1:100])

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed